In [33]:
import pandas as pd
import pyodbc
import statsmodels.api as sm
from scipy import stats

In [35]:
connection_string = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=DESKTOP-FEC13P5\SQLEXPRESS;'
    r'DATABASE=ABC_Pharmaceutical;'
    r'Trusted_Connection=yes;'
)

In [37]:
conn = pyodbc.connect(connection_string)

In [39]:
query = "SELECT * FROM sales_main"
sales_data = pd.read_sql(query, conn)

C:\Users\ZINOX\AppData\Local\Temp\ipykernel_13976\3474255200.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_data = pd.read_sql(query, conn)


In [42]:
sales_data.dropna(inplace=True)

In [24]:
print(sales_data.head())

  SalesID Distributor                      CustomerName        City  Country  \
0  152048     Koss                Erdman PLC Pharma Plc     Freital  Germany   
1  152048     Koss                Erdman PLC Pharma Plc     Freital  Germany   
2  152049     Koss     Hermiston LLC Pharmaceutical Ltd  Weingarten  Germany   
3  152049     Koss     Hermiston LLC Pharmaceutical Ltd  Weingarten  Germany   
4  152049     Koss     Hermiston LLC Pharmaceutical Ltd  Weingarten  Germany   

  Latitude Longitude  ProductID  ChannelID   Channel           Product Name  \
0  51.0167     13.65      238.0        4.0  Hospital  Cephozumab Synmethate   
1  51.0167     13.65      238.0        4.0  Hospital  Cephozumab Synmethate   
2  47.8078    9.6417       59.0        2.0  Pharmacy  Alemtuvatol Megalinum   
3  47.8078    9.6417       59.0        2.0  Pharmacy  Alemtuvatol Megalinum   
4  47.8078    9.6417       59.0        2.0  Pharmacy  Alemtuvatol Megalinum   

  Quantity  Price     Sales    Month        

In [48]:
print("Data types before conversion:")
print(sales_data.dtypes)

Data types before conversion:
SalesID          object
Distributor      object
CustomerName     object
City             object
Country          object
Latitude         object
Longitude        object
ProductID       float64
ChannelID       float64
Channel          object
Product Name     object
Quantity         object
Price           float64
Sales           float64
Month            object
Year             object
SalesRepID      float64
Manager          object
dtype: object


In [54]:
sales_data['Price'] = pd.to_numeric(sales_data['Price'], errors='coerce')
sales_data['Quantity'] = pd.to_numeric(sales_data['Quantity'], errors='coerce')



In [56]:
correlation, p_value_h1 = stats.pearsonr(sales_data['Price'], sales_data['Quantity'])
print(f"Hypothesis 1: Correlation coefficient: {correlation:.4f}, P-value: {p_value_h1:.4f}")



Hypothesis 1: Correlation coefficient: -0.0012, P-value: 0.2031


In [60]:
hospital_sales = sales_data[sales_data['Channel'] == 'Hospital']['Quantity']
pharmacy_sales = sales_data[sales_data['Channel'] == 'Pharmacy']['Quantity']
t_stat, p_value_h2 = stats.ttest_ind(hospital_sales, pharmacy_sales, equal_var=False)
print(f"Hypothesis 2: T-statistic: {t_stat:.4f}, P-value: {p_value_h2:.4f}")

Hypothesis 2: T-statistic: -6.8860, P-value: 0.0000


In [68]:
urban_sales = sales_data[sales_data['City'] == 'Sales']['Quantity']
rural_sales = sales_data[sales_data['City'] == 'Sales']['Quantity']
t_stat_h3, p_value_h3 = stats.ttest_ind(urban_sales, rural_sales, equal_var=False)
print(f"Hypothesis 3: T-statistic: {t_stat_h3:.4f}, P-value: {p_value_h3:.4f}")

Hypothesis 3: T-statistic: nan, P-value: nan


In [70]:
alpha = 0.05
if p_value_h1 < alpha:
    print("Reject Hypothesis 1: There is a significant correlation between product prices and sales quantities.")
else:
    print("Fail to reject Hypothesis 1: No significant correlation found.")

if p_value_h2 < alpha:
    print("Reject Hypothesis 2: Sales performance varies significantly between Hospital and Pharmacy channels.")
else:
    print("Fail to reject Hypothesis 2: No significant difference found.")

if p_value_h3 < alpha:
    print("Reject Hypothesis 3: Sales are significantly higher in urban areas compared to rural areas.")
else:
    print("Fail to reject Hypothesis 3: No significant difference found.")

Fail to reject Hypothesis 1: No significant correlation found.
Reject Hypothesis 2: Sales performance varies significantly between Hospital and Pharmacy channels.
Fail to reject Hypothesis 3: No significant difference found.
